<b>Google Colab</b> <a href="https://colab.research.google.com/github/kirillzyusko/deeplearning/blob/master/7/lab.ipynb">link</a>

Authorize google + kaggle

In [1]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


Be sure, that we authorized and have an access to kaggle

In [2]:
%ls /content/.kaggle/

kaggle.json*


# **Part 1: Download dataset, extract**

Download dataset:

In [3]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content/kaggle/imdb

 35% 9.00M/25.7M [00:00<00:01, 11.7MB/s]
100% 25.7M/25.7M [00:00<00:00, 35.2MB/s]


Extract .zip

In [4]:
!unzip kaggle/imdb/imdb-dataset-of-50k-movie-reviews.zip -d data

Archive:  kaggle/imdb/imdb-dataset-of-50k-movie-reviews.zip
  inflating: data/IMDB Dataset.csv   


Files:

In [5]:
%ls data

'IMDB Dataset.csv'


Read data using pandas:

In [50]:
import pandas as pd

df = pd.read_csv('data/IMDB Dataset.csv')
print(df.shape)

df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Imports

In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [0]:
dictionary_length = 10000
input_length = 100

tokenizer = Tokenizer(num_words=dictionary_length)
tokenizer.fit_on_texts(df.review.values)

In [122]:
post_seq = tokenizer.texts_to_sequences(df.review.values)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 124252 unique tokens.


In [0]:
post_seq_padded = pad_sequences(post_seq, maxlen=input_length)

In [0]:
x_original = post_seq_padded
x_original = np.array(x_original)

y_original = df['sentiment'].replace({ 'positive': 1, 'negative': 0 }).values
y_original = np.array(y_original)

x, y = shuffle(x_original, y_original, random_state=23)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# **Part 2: RNN with LSTM**

In [113]:
model = Sequential()
model.add(Embedding(dictionary_length, 8, input_length=input_length))
model.add(Bidirectional(LSTM(16, return_sequences=False))) # dropout=0.2, recurrent_dropout=0.2
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 8)            80000     
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 32)                3200      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 33        
Total params: 83,233
Trainable params: 83,233
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [115]:
model.fit(x=x_train, y=y_train, batch_size=256, verbose=1, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
79/79 [==============================] - 2s 22ms/step - loss: 0.6516 - accuracy: 0.6165 - val_loss: 0.4897 - val_accuracy: 0.7408
Epoch 2/10
79/79 [==============================] - 1s 14ms/step - loss: 0.3705 - accuracy: 0.8431 - val_loss: 0.4038 - val_accuracy: 0.8268
Epoch 3/10
79/79 [==============================] - 1s 15ms/step - loss: 0.2742 - accuracy: 0.8924 - val_loss: 0.3578 - val_accuracy: 0.8528
Epoch 4/10
79/79 [==============================] - 1s 15ms/step - loss: 0.2252 - accuracy: 0.9172 - val_loss: 0.4646 - val_accuracy: 0.8116
Epoch 5/10
79/79 [==============================] - 1s 15ms/step - loss: 0.1945 - accuracy: 0.9288 - val_loss: 0.4152 - val_accuracy: 0.8450
Epoch 6/10
79/79 [==============================] - 1s 15ms/step - loss: 0.1591 - accuracy: 0.9445 - val_loss: 0.4120 - val_accuracy: 0.8424
Epoch 7/10
79/79 [==============================] - 1s 15ms/step - loss: 0.1351 - accuracy: 0.9560 - val_loss: 0.4886 - val_accuracy: 0.8356
Epoch 8/10
79

In [116]:
model.evaluate(x_test, y_test)

782/782 [==============================] - 4s 6ms/step - loss: 0.5428 - accuracy: 0.8340


[0.542818009853363, 0.8339599967002869]

# **Part 3: Using GloVe**

In [118]:
!wget "http://nlp.stanford.edu/data/glove.6B.zip"

--2020-04-03 21:06:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-03 21:06:26--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-03 21:06:26--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [119]:
!mkdir glove
!unzip glove.6B.zip -d glove

Archive:  glove.6B.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  


In [120]:
glove_dir = 'glove'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embedding_dim = 100

embedding_matrix = np.zeros((dictionary_length, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < dictionary_length:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [126]:
model = Sequential()
model.add(Embedding(dictionary_length, embedding_dim, input_length=input_length))
model.add(Bidirectional(LSTM(16))) # dropout=0.2, recurrent_dropout=0.2
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_val, y_val))
score, acc = model.evaluate(x_test, y_test)

print('Test accuracy:', acc)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 100)          1000000   
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 32)                14976     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 33        
Total params: 1,015,009
Trainable params: 1,015,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 8s 12ms/step - loss: 0.5689 - accuracy: 0.6964 - val_loss: 0.4867 - val_accuracy: 0.7644
Epoch 2/10
625/625 [==============================] - 7s 11ms/step - loss: 0.4412 - accuracy: 0.7943 - val_loss: 0.4050 - val_accuracy: 0.8112
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.4041 - acc

# **Part 4: New NN architecture**

In [127]:
model = Sequential()
model.add(Embedding(dictionary_length, embedding_dim, input_length=input_length))
model.add(Bidirectional(LSTM(32, return_sequences=True))) # dropout=0.2, recurrent_dropout=0.2
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_val, y_val))
score, acc = model.evaluate(x_test, y_test)

print('Test accuracy:', acc)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 100, 100)          1000000   
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 100, 64)           34048     
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 100, 64)           24832     
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 64)                24832     
_________________________________________________________________
dense_20 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_21 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_22 (Dense)             (None, 1)               

# **Part 5: DeepMoji**

???

In [0]:
!pip3 install torch==1.0.1 -f https://download.pytorch.org/whl/cpu/stable 
!git clone https://github.com/huggingface/torchMoji
import os
os.chdir('torchMoji')
!pip3 install -e .

In [0]:
!python3 scripts/download_weights.py

In [0]:
"""Trains the DeepMoji architecture on the IMDB sentiment classification task.
   This is a simple example of using the architecture without the pretrained model.
   The architecture is designed for transfer learning - it should normally
   be used with the pretrained model for optimal performance.
"""
from __future__ import print_function
import example_helper
import numpy as np
from keras.preprocessing import sequence
from keras.datasets import imdb
from deepmoji.model_def import deepmoji_architecture

# Seed for reproducibility
np.random.seed(1337)

nb_tokens = 20000
maxlen = 80
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=nb_tokens)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = deepmoji_architecture(nb_classes=2, nb_tokens=nb_tokens, maxlen=maxlen)
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=15,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)